- I tried to augment the data using duplicates.

# Import

In [ ]:
from collections import Counter

import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

# Configs

In [ ]:
config = {
    "TRAIN_PATH" : "../input/jigsaw-toxic-severity-rating/validation_data.csv"
}

# About Data

In [ ]:
train = pd.read_csv(config["TRAIN_PATH"])

In [ ]:
train.head(3)

In [ ]:
train.shape

In [ ]:
train_texts = train["less_toxic"].tolist() + train["more_toxic"].tolist()
print("Train texts: {} | unique: {}".format(len(train_texts), len(set(train_texts))))

In [ ]:
train_texts_counter = Counter(train_texts)
train_texts_counter.most_common()[:4]

In [ ]:
sns.histplot([count for text, count in train_texts_counter.most_common()],
             discrete=True)

There are duplicates texts in the training data pairs.
It may help to augment training data.

# Augmentation

- If A<B and B<C, then A<C  (Ideally)

In [ ]:
anker_set = set(train["more_toxic"].tolist()) & set(train["less_toxic"].tolist())

In [ ]:
print("anker: {}".format(len(anker_set)))

In [ ]:
anker_pair = {"less_toxic": [], "more_toxic": []}
for anker in tqdm(anker_set):
    less_texts = set(train.loc[train["more_toxic"]==anker, "less_toxic"])
    more_texts = set(train.loc[train["less_toxic"]==anker, "more_toxic"])
    for less_text in less_texts:
        for more_text in more_texts:
            if less_text == more_text:
                continue
            anker_pair["less_toxic"].append(less_text)
            anker_pair["more_toxic"].append(more_text)
anker_pair["worker"] = [-1] * len(anker_pair["less_toxic"])

In [ ]:
anker_pair = pd.DataFrame(anker_pair)
anker_pair.head(5)

- There are pairs where A<B and B<A.
- So, maybe more data will only confuse the model.

In [ ]:
train_augmented = pd.concat([train, anker_pair]).reset_index(drop=True)
print("Train: {} + Augment: {} -> All: {}".format(len(train), len(anker_pair), len(train_augmented)))

In [ ]:
train_augmented.head(5)

In [ ]:
train_augmented.to_csv("train_augmented.csv", index=False)